In [1]:
# install the necessary dependencies for your projects

# pip install opencv-python opencv-contrib-python
# pip install opencv-python
# pip install dlib
# pip install numpy
# pip install scipy
# pip install imutils
# pip install cmake
# pip install dlib
# pip install mediapipe

In [2]:
import cv2
import numpy as np
import mediapipe as mp
from scipy.spatial import distance as dist
import time
from datetime import datetime


In [3]:
import cv2
import numpy as np
import mediapipe as mp
from scipy.spatial import distance as dist
import time
from datetime import datetime
import threading

class ExamProctor:
    def __init__(self, alert_callback=None):
        # Initialize Mediapipe Face Mesh
        self.mp_face_mesh = mp.solutions.face_mesh
        self.face_mesh = self.mp_face_mesh.FaceMesh(
            max_num_faces=2,
            min_detection_confidence=0.5,
            min_tracking_confidence=0.5
        )
        self.mp_drawing = mp.solutions.drawing_utils

        # Thresholds and counters
        self.EYE_AR_THRESH = 0.25
        self.EYE_AR_CONSEC_FRAMES = 20
        self.SUSPICIOUS_MOVEMENT_THRESH = 50
        self.counter = 0
        self.alerts = []
        
        # Video capture
        self.cap = cv2.VideoCapture(0)
        
        # Movement tracking
        self.prev_position = None
        self.suspicious_count = 0
        
        # GUI integration
        self.alert_callback = alert_callback  # Callback to update GUI alerts
        self.running = False

    def eye_aspect_ratio(self, eye_landmarks, landmarks, image_shape):
        """Calculate Eye Aspect Ratio (EAR)"""
        h, w = image_shape[:2]
        eye_coords = [(int(landmarks[p].x * w), int(landmarks[p].y * h)) for p in eye_landmarks]
        A = dist.euclidean(eye_coords[1], eye_coords[5])
        B = dist.euclidean(eye_coords[2], eye_coords[4])
        C = dist.euclidean(eye_coords[0], eye_coords[3])
        ear = (A + B) / (2.0 * C)
        return ear

    def detect_suspicious_behavior(self, frame):
        """Detect suspicious behavior and update alerts"""
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = self.face_mesh.process(frame_rgb)
        h, w = frame.shape[:2]

        if not results.multi_face_landmarks:
            alert = f"[{datetime.now()}] No face detected"
            self.alerts.append(alert)
            if self.alert_callback:
                self.alert_callback(alert)
            cv2.putText(frame, "NO FACE DETECTED!", (10, 90),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            return frame
        elif len(results.multi_face_landmarks) > 1:
            alert = f"[{datetime.now()}] Multiple faces detected"
            self.alerts.append(alert)
            if self.alert_callback:
                self.alert_callback(alert)
            cv2.putText(frame, "MULTIPLE FACES DETECTED!", (10, 90),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            return frame

        face_landmarks = results.multi_face_landmarks[0].landmark
        LEFT_EYE = [33, 160, 158, 133, 153, 144]
        RIGHT_EYE = [362, 385, 387, 263, 373, 380]

        # Eye tracking
        leftEAR = self.eye_aspect_ratio(LEFT_EYE, face_landmarks, frame.shape)
        rightEAR = self.eye_aspect_ratio(RIGHT_EYE, face_landmarks, frame.shape)
        ear = (leftEAR + rightEAR) / 2.0

        if ear < self.EYE_AR_THRESH:
            self.counter += 1
            if self.counter >= self.EYE_AR_CONSEC_FRAMES:
                alert = f"[{datetime.now()}] Potential cheating: Eyes closed"
                self.alerts.append(alert)
                if self.alert_callback:
                    self.alert_callback(alert)
                cv2.putText(frame, "EYES CLOSED!", (10, 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        else:
            self.counter = 0

        # Head movement detection
        nose_tip = (int(face_landmarks[1].x * w), int(face_landmarks[1].y * h))
        if self.prev_position is not None:
            movement = dist.euclidean(self.prev_position, nose_tip)
            if movement > self.SUSPICIOUS_MOVEMENT_THRESH:
                self.suspicious_count += 1
                if self.suspicious_count > 5:
                    alert = f"[{datetime.now()}] Suspicious head movement detected"
                    self.alerts.append(alert)
                    if self.alert_callback:
                        self.alert_callback(alert)
                    cv2.putText(frame, "SUSPICIOUS MOVEMENT!", (10, 60),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            else:
                self.suspicious_count = max(0, self.suspicious_count - 1)
        self.prev_position = nose_tip

        # Visualize eye regions
        for idx in LEFT_EYE + RIGHT_EYE:
            pt = (int(face_landmarks[idx].x * w), int(face_landmarks[idx].y * h))
            cv2.circle(frame, pt, 2, (0, 255, 0), -1)

        return frame

    def run(self):
        """Run proctoring in a loop"""
        self.running = True
        while self.running:
            ret, frame = self.cap.read()
            if not ret:
                print("Failed to capture video")
                break

            frame = self.detect_suspicious_behavior(frame)
            cv2.imshow("Exam Proctor", frame)
            
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        self.cleanup()

    def stop(self):
        """Stop the proctoring loop"""
        self.running = False

    def cleanup(self):
        """Clean up resources and save alerts"""
        self.cap.release()
        cv2.destroyAllWindows()
        self.face_mesh.close()
        
        with open("proctoring_log.txt", "w") as f:
            f.write("\n".join(self.alerts))
        print("Proctoring ended. Alerts saved to proctoring_log.txt")

    def __del__(self):
        self.cleanup()

# Tkinter Front-End
import tkinter as tk
from tkinter import ttk, scrolledtext

class ProctorGUI:
    def __init__(self, root):
        self.root = root
        self.root.title("ProctorAI")
        self.root.geometry("400x500")
        self.root.resizable(False, False)

        # Proctor instance
        self.proctor = ExamProctor(self.update_alerts)

        # Title
        self.title_label = ttk.Label(root, text="ProctorAI", font=("Arial", 16, "bold"))
        self.title_label.pack(pady=10)

        # Start/Stop Buttons
        self.start_button = ttk.Button(root, text="Start Proctoring", command=self.start_proctoring)
        self.start_button.pack(pady=5)

        self.stop_button = ttk.Button(root, text="Stop Proctoring", command=self.stop_proctoring, state="disabled")
        self.stop_button.pack(pady=5)

        # Alerts Display
        self.alert_label = ttk.Label(root, text="Real-Time Alerts", font=("Arial", 12))
        self.alert_label.pack(pady=5)

        self.alert_text = scrolledtext.ScrolledText(root, width=40, height=15, wrap=tk.WORD)
        self.alert_text.pack(pady=5)

        # Log Button
        self.log_button = ttk.Button(root, text="View Log", command=self.view_log)
        self.log_button.pack(pady=5)

    def update_alerts(self, alert):
        """Callback to update the GUI with real-time alerts"""
        self.alert_text.insert(tk.END, alert + "\n")
        self.alert_text.see(tk.END)

    def start_proctoring(self):
        """Start the proctoring in a separate thread"""
        self.start_button.config(state="disabled")
        self.stop_button.config(state="normal")
        self.alert_text.delete(1.0, tk.END)  # Clear previous alerts
        self.proctor_thread = threading.Thread(target=self.proctor.run)
        self.proctor_thread.start()

    def stop_proctoring(self):
        """Stop the proctoring"""
        self.proctor.stop()
        self.proctor_thread.join()
        self.start_button.config(state="normal")
        self.stop_button.config(state="disabled")

    def view_log(self):
        """Display the contents of the log file"""
        try:
            with open("proctoring_log.txt", "r") as f:
                log_content = f.read()
            self.alert_text.delete(1.0, tk.END)
            self.alert_text.insert(tk.END, "Log File Contents:\n\n" + log_content)
        except FileNotFoundError:
            self.alert_text.delete(1.0, tk.END)
            self.alert_text.insert(tk.END, "No log file found yet.")

    def on_closing(self):
        """Handle window close"""
        if self.proctor.running:
            self.proctor.stop()
            self.proctor_thread.join()
        self.root.destroy()

if __name__ == "__main__":
    root = tk.Tk()
    app = ProctorGUI(root)
    root.protocol("WM_DELETE_WINDOW", app.on_closing)
    root.mainloop()

Proctoring ended. Alerts saved to proctoring_log.txt
